# Example of adaptive Lasso

Author: Alexandre Gramfort

Adaptive lasso consists in computing many Lasso with feature
reweighting. It's also known as iterated or reweighted L1.

References:

- Enhancing Sparsity by Reweighted L1 Minimization, Emmanuel J. Candes, Michael B. Wakin, Stephen P. Boyd, https://arxiv.org/abs/0711.1612
- D. Wipf and S. Nagarajan, "Iterative Reweighted $\ell_1$ and $\ell_2$ Methods for Finding Sparse Solutions," in IEEE Journal of Selected Topics in Signal Processing, vol. 4, no. 2, pp. 317-329, April 2010. https://ieeexplore.ieee.org/document/5419071

See also : https://arxiv.org/abs/0807.0575 for reweighting based on L2.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import Lasso

data = fetch_openml("leukemia")
X = data.data.astype(float)
y = 2 * (data.target == data.target[0]).astype(float) - 1.
X /= np.sum(X ** 2, axis=0)  # scale features

In [ ]:
if True:  # g(x) = sqrt(abs(x))
    g = lambda w: np.sqrt(np.abs(w))
    gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)
else: # g(x) = log(eps + abs(x))
    # TODO
    eps = 0.01
    g = None
    gprime = None

n_samples, n_features = X.shape
p_obj = lambda w: 1. / (2 * n_samples) * np.sum((y - np.dot(X, w)) ** 2) \
                  + alpha * np.sum(g(w))

weights = np.ones(n_features)
n_lasso_iterations = 10
alpha = 5e-6

objective = []
for k in range(n_lasso_iterations):
    X_w = X / weights[np.newaxis, :]
    clf = Lasso(alpha=alpha, fit_intercept=False)
    clf.fit(X_w, y)
    coef_ = clf.coef_ / weights
    weights = gprime(coef_)
    objective.append(p_obj(coef_))
    print('Sparsity: %1.3f %%- Objective: %.5f' % (100. * np.mean(coef_ != 0.0), objective[-1]))

In [ ]:
plt.plot(objective)
plt.xlabel('Iterations')
plt.ylabel('Objective');

<div class="alert alert-success">
    <b>EXERCISE:</b>
     <ul>
      <li>Implement reweighting L1 for $g(x) = \log(\epsilon + |x|)$ by <a href="https://arxiv.org/abs/0711.1612">Candes et al.</a></li>
    </ul>
</div>